In [1]:
# we have already stored this in seperate file
import torch
from torch.utils.data import Dataset


class RatingsTrainDataset(Dataset):

    def __init__(self, ratings, all_product_ids):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_product_ids)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_product_ids):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['user_id_int'], ratings['product_id_int']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_product_ids)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_product_ids)
                users.append(u)
                items.append(negative_item)
                labels.append(0)
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [2]:
# we have already stored this in seperate file
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class NCF(pl.LightningModule):
    
    def __init__(self, num_users, num_items, ratings, all_product_ids):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_product_ids = all_product_ids
        
    def forward(self, user_input, item_input):
        
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(RatingsTrainDataset(self.ratings, self.all_product_ids),
                          batch_size=512, num_workers=0)

In [3]:
import pandas as pd
import numpy as np
import pickle
import torch
import os

def get_collaborative_recommendations(user_id, company_id, offset, limit):
    
    if not os.path.exists(company_id):
        return []
    
    original_user_file = open(f'{company_id}/user_mapping.pkl', 'rb')
    original_user_ids = pickle.load(original_user_file)
    
    if user_id not in original_user_ids.keys():
        return [];
    else:
        product_ids_file = open(f'{company_id}/reverse_product_mapping.pkl', 'rb')
        reverse_product_mapping = pickle.load(product_ids_file)
        model_file = open(f'{company_id}/collaborative_filtering_model.pkl', 'rb')
        model_data = pickle.load(model_file)
        user_id = original_user_ids[user_id]
        model = model_data['model']
        all_product_ids = model_data['product_ids']
        user_interacted_items = model_data['interacted_items']
        interacted_items = user_interacted_items[user_id]
        not_interacted_items = list(set(all_product_ids) - set(interacted_items))
        predicted_labels = np.squeeze(model(torch.tensor([user_id]*len(not_interacted_items)), torch.tensor(not_interacted_items)).detach().numpy())
        items = [not_interacted_items[i] for i in np.argsort(predicted_labels)[::-1].tolist()]
        if len(items) < offset+1:
            return []
        return [reverse_product_mapping.get(x) for x in items[offset : offset+limit if offset+limit <= len(items) else len(items)]]

In [5]:
get_collaborative_recommendations('A3J3BRHTDRFJ2G', 'user_0', 0, 1000)

['B005HMKKH4',
 'B009SYZ8OC',
 'B000LRMS66',
 'B0019EHU8G',
 'B0041Q38NU',
 'B00DR0PDNE',
 'B003ES5ZUU',
 'B00B588HY2',
 'B002V88HFE',
 'B007WTAJTO',
 'B002QEBMAK',
 'B001TH7GSW',
 'B005CLPP84',
 'B000A6PPOK',
 'B00622AG6S',
 'B00834SJNA',
 'B0036Q7MV0',
 'B003ELYQGG',
 'B006GWO5WK',
 'B00BGGDVOO',
 'B000MX48VM',
 'B0034CL2ZI',
 'B002WE6D44',
 'B007R5YDYA',
 'B006W8U2MU',
 'B0002L5R78',
 'B002BH3I9U',
 'B0001FTVEK',
 'B000JMJWV2',
 'B004G6002M',
 'B004QK7HI8',
 'B000CKVOOY',
 'B002MAPRYU',
 'B004GCJEZU',
 'B001XURP7W',
 'B0052SCU8U',
 'B000QUUFRW',
 'B00BQ4F9ZA',
 'B0044YU60M',
 'B00004ZCJE',
 'B000OGX5AM',
 'B002RM08RE',
 'B000FBK3QK',
 'B000VX6XL6',
 'B004W2JKWG',
 'B005FYNSPK',
 'B004LSNF04',
 'B00316263Y',
 'B007I5JT4S',
 'B00007E7JU',
 'B004T9RR6I',
 'B002SZEOLG',
 'B000Q8UAWY',
 'B0048IW030',
 'B003FVVMS0',
 'B00B46XUQU',
 'B00004T8R2',
 'B000N99BBC',
 'B000JE7GPY',
 'B0012S4APK',
 'B000S5Q9CA',
 'B005DKZTMG',
 'B004GF8TIK',
 'B002VPE1WK',
 'B000RQSGYO',
 'B006EKIQYM',
 'B004XC6G